In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
import math
torch.manual_seed(8)
import time
import numpy as np
import gc
import sys
sys.setrecursionlimit(50000)
import pickle
torch.backends.cudnn.benchmark = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')
# from tensorboardX import SummaryWriter
torch.nn.Module.dump_patches = True
import copy
import pandas as pd
#then import my own modules
from AttentiveFP.AttentiveLayers_Sim import Fingerprint
from AttentiveFP import Fingerprint_viz, save_smiles_dicts, get_smiles_dicts, get_smiles_array, moltosvg_highlight

In [2]:
from rdkit import Chem
# from rdkit.Chem import AllChem
from rdkit.Chem import QED
from rdkit.Chem import rdMolDescriptors, MolSurf
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
%matplotlib inline
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
import seaborn as sns; sns.set()
from IPython.display import SVG, display
import sascorer
from AttentiveFP.utils import EarlyStopping
from AttentiveFP.utils import Meter

In [3]:
raw_filename = "../data/active_inactive/median_active/Kd/P47747.csv" #目标域active
test_active = 50
train_rate = 1
val_rate = 0.2
random_seed = 68
inactive_filename = "../data/active_inactive/median_inactive/Kd/P47747.csv"#目标域inactive
file_list1 = raw_filename.split('/')
file1 = file_list1[-2]+'_'+file_list1[-1]
file1 = file1[:-4]
model_file = "model_file/1_sim_"+file1+"_theorem"
print(model_file)

model_file/1_sim_Kd_P47747_theorem


In [4]:
# task_name = 'Malaria Bioactivity'
tasks = ['value']

# raw_filename = "../data/active_inactive/median_active/EC50/Q99500.csv"
feature_filename = raw_filename.replace('.csv','.pickle')
filename = raw_filename.replace('.csv','')
prefix_filename = raw_filename.split('/')[-1].replace('.csv','')
smiles_tasks_df = pd.read_csv(raw_filename, header=0, names = ["smiles","value"],usecols=[3,8])
# smiles_tasks_df = smiles_tasks_df[1:]
# smiles_tasks_df = smiles_tasks_df.drop(0,axis=1,inplace=False) 
print(smiles_tasks_df[:5])
# print(smiles_tasks_df.iloc(1))
def add_canonical_smiles(smiles_tasks_df):
    smilesList = smiles_tasks_df.smiles.values
    print("number of all smiles: ",len(smilesList))
    atom_num_dist = []
    remained_smiles = []
    canonical_smiles_list = []
    for smiles in smilesList:
        try:        
            mol = Chem.MolFromSmiles(smiles)
            atom_num_dist.append(len(mol.GetAtoms()))
            remained_smiles.append(smiles)
            canonical_smiles_list.append(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True))
        except:
            print(smiles)
            pass
    print("number of successfully processed smiles: ", len(remained_smiles))
    smiles_tasks_df = smiles_tasks_df[smiles_tasks_df["smiles"].isin(remained_smiles)]
    smiles_tasks_df['cano_smiles'] =canonical_smiles_list
    return smiles_tasks_df
# print(smiles_tasks_df)
smiles_tasks_df = add_canonical_smiles(smiles_tasks_df)

print(smiles_tasks_df.head())
# plt.figure(figsize=(5, 3))
# sns.set(font_scale=1.5)
# ax = sns.distplot(atom_num_dist, bins=28, kde=False)
# plt.tight_layout()
# # plt.savefig("atom_num_dist_"+prefix_filename+".png",dpi=200)
# plt.show()
# plt.close()


                                              smiles     value
0  C1CCN(CC1)CC2=CC(=CC=C2)OCCCN=C(NCCCCNC(=O)C3=... -3.950000
1                        CC1=C(N=CN1)CSCCNC(=NC)NC#N -2.600003
2           C1=C(N=C(S1)N=C(N)N)CSCCC(=NS(=O)(=O)N)N -1.771587
3  C1CCN(CC1)CC2=CC(=CC=C2)OCCCN=C(NCCNC(=O)C3=C4... -2.260000
4                CC1=C(N=CN1)CSCCNC(=NCCCC2=NC=CS2)N -2.500003
number of all smiles:  105
number of successfully processed smiles:  105
                                              smiles     value  \
0  C1CCN(CC1)CC2=CC(=CC=C2)OCCCN=C(NCCCCNC(=O)C3=... -3.950000   
1                        CC1=C(N=CN1)CSCCNC(=NC)NC#N -2.600003   
2           C1=C(N=C(S1)N=C(N)N)CSCCC(=NS(=O)(=O)N)N -1.771587   
3  C1CCN(CC1)CC2=CC(=CC=C2)OCCCN=C(NCCNC(=O)C3=C4... -2.260000   
4                CC1=C(N=CN1)CSCCNC(=NCCCC2=NC=CS2)N -2.500003   

                                         cano_smiles  
0  N#CNC(=NCCCOc1cccc(CN2CCCCC2)c1)NCCCCNC(=O)c1c...  
1                         CN=C(NC#N)NCCSCc

In [5]:
start_time = str(time.ctime()).replace(':','-').replace(' ','_')

p_dropout= 0.03
fingerprint_dim = 100

weight_decay = 4.3 # also known as l2_regularization_lambda
learning_rate = 4
radius = 2
T = 1
per_task_output_units_num = 1 # for regression model
output_units_num = len(tasks) * per_task_output_units_num

In [6]:
# remained_df[tasks[0]] = remained_df[tasks[0]].astype(float)
# test_df = remained_df.sample(frac=0.8,random_state=random_seed) #随机取0.8做test

# smiles_tasks_df 为读取出来的数据
# tmp = smiles_tasks_df.loc[smiles_tasks_df['value'].str.contains('-')]
# print(tmp.shape)
# smiles_tasks_df = smiles_tasks_df.drop(tmp.index)
sample_df = smiles_tasks_df.sample(frac=train_rate,random_state=random_seed)

# sample_df["value"] = sample_df["value"].astype(float) #astype()强制转换数据类型
# sample_df["value"] = sample_df["value"].apply(lambda x:-math.log10(x))
# print(sample_df[:10])

# test_active = 200
test_df = sample_df.sample(test_active,random_state=random_seed)
train_df = sample_df.drop(test_df.index) 
print(train_df.shape,sample_df.shape,test_df.shape)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
# print(train_df.head())
# print(test_df.head())
# print(len(test_df),sorted(test_df.cano_smiles.values))

(55, 3) (105, 3) (50, 3)


In [7]:
# inactive_filename = "../data/active_inactive/median_inactive/EC50/Q99500.csv"
inactive_df = pd.read_csv(inactive_filename,header=0,names=["smiles"],usecols=[3])
inactive_df = add_canonical_smiles(inactive_df)
inactive_df['value']=-4.1
print(inactive_df.head())
for l in inactive_df["cano_smiles"]:
    if l in sample_df["cano_smiles"]:
        print("same smiles:",l)
        
test_df = pd.concat([test_df,inactive_df],axis=0)
test_df = test_df.reset_index(drop=True)
print(test_df.shape)
print(test_df.head())

number of all smiles:  94
number of successfully processed smiles:  94
                                            smiles  \
0      CC(=O)C1=CC=C(C=C1)OCC2=CC=C(C=C2)CN3CCCCC3   
1  CC(C1=CC=C(C=C1)OCCCC2=CN=CN2)NO.C(=O)(C(=O)O)O   
2                      CN=C(N)NCCCCC1=CN=CN1.Cl.Cl   
3                         C1CCC(CC1)CCCOCC2=CN=CN2   
4                               CCCCCOCCCC1=CN=CN1   

                                   cano_smiles  value  
0        CC(=O)c1ccc(OCc2ccc(CN3CCCCC3)cc2)cc1   -4.1  
1  CC(NO)c1ccc(OCCCc2cnc[nH]2)cc1.O=C(O)C(=O)O   -4.1  
2                 CN=C(N)NCCCCc1cnc[nH]1.Cl.Cl   -4.1  
3                    c1ncc(COCCCC2CCCCC2)[nH]1   -4.1  
4                          CCCCCOCCCc1cnc[nH]1   -4.1  
(144, 3)
                                              smiles     value  \
0                        CC1=C(N=CN1)CSCCNC(=NC)NC#N -2.600003   
1  C1CCN(CC1)CC2=CC(=CC=C2)OCCCN=C(NCCCCCCCCNC3=C... -1.039811   
2             CC(C)CN=CNC1=CC=C(C=C1)C2=CN=CN2.Cl.Cl -0.71999

In [8]:
print(feature_filename)
print(filename)
total_df = pd.concat([train_df,test_df],axis=0)
total_smilesList = total_df['smiles'].values
print(len(total_smilesList))
# if os.path.isfile(feature_filename):
#     feature_dicts = pickle.load(open(feature_filename, "rb" ))
# else:
#     feature_dicts = save_smiles_dicts(smilesList,filename)
feature_dicts = save_smiles_dicts(total_smilesList,filename)
remained_df = total_df[total_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
uncovered_df = total_df.drop(remained_df.index)
uncovered_df

../data/active_inactive/median_active/Kd/P47747.pickle
../data/active_inactive/median_active/Kd/P47747
199
feature dicts file saved as ../data/active_inactive/median_active/Kd/P47747.pickle


,smiles,value,cano_smiles


In [9]:
name = file1+'|'+str(train_rate)+'_'+str(test_active)
print(name)
train_df.to_csv('../data/active_inactive/benchmarks/Sim/'+name+'_train.csv', sep=',', header=True, index=False)
test_df.to_csv('../data/active_inactive/benchmarks/Sim/'+name+'_test.csv',sep= ',', header=True, index=False)
val_df = train_df.sample(frac=val_rate,random_state=random_seed)
train_df = train_df.drop(val_df.index)
train_df = train_df.reset_index(drop=True)
train_df = train_df[train_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
train_df = train_df.reset_index(drop=True)
val_df = val_df[val_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
val_df = val_df.reset_index(drop=True)
test_df = test_df[test_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
test_df = test_df.reset_index(drop=True)
print(train_df.shape,test_df.shape,val_df.shape)

Kd_P47747|1_50
(44, 3) (144, 3) (11, 3)


In [10]:
x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array([total_df["cano_smiles"].values[0]],feature_dicts)
print('x_atom',x_atom,'x_bonds', x_bonds, 'x_atom_index',x_atom_index, 'x_bond_index',x_bond_index,
      'x_mask',x_mask, 'smiles_to_rdkit_list',smiles_to_rdkit_list)
num_atom_features = x_atom.shape[-1]
num_bond_features = x_bonds.shape[-1]
loss_function = nn.MSELoss()
model = Fingerprint(radius, T, num_atom_features, num_bond_features,
            fingerprint_dim, output_units_num, p_dropout)
model.cuda()

# optimizer = optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)
optimizer = optim.Adam(model.parameters(), 10**-learning_rate, weight_decay=10**-weight_decay)
# optimizer = optim.SGD(model.parameters(), 10**-learning_rate, weight_decay=10**-weight_decay)

# tensorboard = SummaryWriter(log_dir="runs/"+start_time+"_"+prefix_filename+"_"+str(fingerprint_dim)+"_"+str(p_dropout))

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
# print(params)
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, param.data.shape)
        

x_atom [[[0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]] x_bonds [[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 1. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 1. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0.

In [11]:
import numpy as np
from matplotlib import pyplot as plt

def sorted_show_pik(dataset, p, k, k_predict, i, acc):
    p_value = dataset[tasks[0]].astype(float).tolist()
    x = np.arange(0,len(dataset),1)
#     print('plt',dataset.head(),p[:10],k_predict,k)
#     plt.figure()
#     fig, ax1 = plt.subplots()
#     ax1.grid(False)
#     ax2 = ax1.twinx()
#     plt.grid(False)
    plt.scatter(x,p,marker='.',s=6,color='r',label='predict')
#     plt.ylabel('predict')
    plt.scatter(x,p_value,s=6,marker=',',color='blue',label='p_value')
    plt.axvline(x=k-1,ls="-",c="black")#添加垂直直线
    k_value = np.ones(len(dataset))
# #     print(EC50[k-1])
    k_value = k_value*k_predict
    plt.plot(x,k_value,'-',color='black')
    plt.ylabel('p_value')
    plt.title("epoch: {},  top-k recall: {}".format(i,acc))
    plt.legend(loc=3,fontsize=5)
    plt.show()
    

def topk_acc2(df, predict, k, active_num, show_flag=False, i=0):
    df['predict'] = predict
    df2 = df.sort_values(by='predict',ascending=False) # 拼接预测值后对预测值进行排序
#     print('df2:\n',df2)
    
    df3 = df2[:k]  #取按预测值排完序后的前k个
    
    true_sort = df.sort_values(by=tasks[0],ascending=False) #返回一个新的按真实值排序列表
    k_true = true_sort[tasks[0]].values[k-1]  # 真实排第k个的活性值
#     print('df3:\n',df3['predict'])
#     print('k_true: ',type(k_true),k_true)
#     print('k_true: ',k_true,'min_predict: ',df3['predict'].values[-1],'index: ',df3['predict'].values>=k_true,'acc_num: ',len(df3[df3['predict'].values>=k_true]),
#           'fp_num: ',len(df3[df3['predict'].values>=-4.1]),'k: ',k)
    acc = len(df3[df3[tasks[0]].values>=k_true])/k #预测值前k个中真实排在前k个的个数/k
    fp = len(df3[df3[tasks[0]].values==-4.1])/k  #预测值前k个中为-4.1的个数/k
    if k>active_num:
        min_active = true_sort[tasks[0]].values[active_num-1]
        acc = len(df3[df3[tasks[0]].values>=min_active])/k
    
    if(show_flag):
        #进来的是按实际活性值排好序的
        sorted_show_pik(true_sort,true_sort['predict'],k,k_predict,i,acc)
    return acc,fp

def topk_recall(df, predict, k, active_num, show_flag=False, i=0):
    df['predict'] = predict
    df2 = df.sort_values(by='predict',ascending=False) # 拼接预测值后对预测值进行排序
#     print('df2:\n',df2)
        
    df3 = df2[:k]  #取按预测值排完序后的前k个，因为后面的全是-4.1
    
    true_sort = df.sort_values(by=tasks[0],ascending=False) #返回一个新的按真实值排序列表
    min_active = true_sort[tasks[0]].values[active_num-1]  # 真实排第k个的活性值
#     print('df3:\n',df3['predict'])
#     print('min_active: ',type(min_active),min_active)
#     print('min_active: ',min_active,'min_predict: ',df3['predict'].values[-1],'index: ',df3['predict'].values>=min_active,'acc_num: ',len(df3[df3['predict'].values>=min_active]),
#           'fp_num: ',len(df3[df3['predict'].values>=-4.1]),'k: ',k,'active_num: ',active_num)
    acc = len(df3[df3[tasks[0]].values>-4.1])/active_num #预测值前k个中真实排在前active_num个的个数/active_num
    fp = len(df3[df3[tasks[0]].values==-4.1])/k  #预测值前k个中为-4.1的个数/active_num
    
    if(show_flag):
        #进来的是按实际活性值排好序的
        sorted_show_pik(true_sort,true_sort['predict'],k,k_predict,i,acc)
    return acc,fp

    
def topk_acc_recall(df, predict, k, active_num, show_flag=False, i=0):
    if k>active_num:
        return topk_recall(df, predict, k, active_num, show_flag, i)
    return topk_acc2(df,predict,k, active_num,show_flag,i)

def weighted_top_index(df, predict, active_num):
    weighted_acc_list=[]
    for k in np.arange(1,len(df)+1,1):
        acc, fp = topk_acc_recall(df, predict, k, active_num)
        weight = (len(df)-k)/len(df)
#         print('weight=',weight,'acc=',acc)
        weighted_acc_list.append(acc*weight)#
    weighted_acc_list = np.array(weighted_acc_list)
#     print('weighted_acc_list=',weighted_acc_list)
    return np.sum(weighted_acc_list)/weighted_acc_list.shape[0]

def AP(df, predict, active_num):
    prec = []
    rec = []
    for k in np.arange(1,len(df)+1,1):
        prec_k, fp1 = topk_acc2(df,predict,k, active_num)
        rec_k, fp2 = topk_recall(df, predict, k, active_num)
        prec.append(prec_k)
        rec.append(rec_k)
    # 取所有不同的recall对应的点处的精度值做平均
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # 计算包络线，从后往前取最大保证precise非减
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # 找出所有检测结果中recall不同的点
    i = np.where(mrec[1:] != mrec[:-1])[0]
#     print(prec)
#     print('prec='+str(prec)+'\n\n'+'rec='+str(rec))

    # and sum (\Delta recall) * prec
    # 用recall的间隔对精度作加权平均
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [12]:
def caculate_r2(y,predict):
#     print(y)
#     print(predict)
    y = torch.FloatTensor(y).reshape(-1,1)
    predict = torch.FloatTensor(predict).reshape(-1,1)
    y_mean = torch.mean(y)
    predict_mean = torch.mean(predict)
    
    y1 = torch.pow(torch.mm((y-y_mean).t(),(predict-predict_mean)),2)
    y2 = torch.mm((y-y_mean).t(),(y-y_mean))*torch.mm((predict-predict_mean).t(),(predict-predict_mean))
#     print(y1,y2)
    return y1/y2

In [13]:
from torch.autograd import Variable
def l2_norm(input, dim):
    norm = torch.norm(input, dim=dim, keepdim=True)
    output = torch.div(input, norm+1e-6)
    return output

def normalize_perturbation(d,dim=-1):
    output = l2_norm(d, dim)
    return output

def tanh(x):
    return (torch.exp(x)-torch.exp(-x))/(torch.exp(x)+torch.exp(-x))

def sigmoid(x):
    return 1/(1+torch.exp(-x))

def perturb_feature(f, pred, model, alpha=1, lamda=10**-learning_rate):
    pred = pred.detach()
    eps = 1e-6 * normalize_perturbation(torch.randn(f.shape))
    eps = Variable(eps, requires_grad=True)
    # Predict on randomly perturbed image
    eps_p = model(feature_only=True, feature1=f, feature2=f+eps.cuda())
    eps_p_ = model(feature_only=True, feature1=f, feature2=f-eps.cuda())
    p_aux = nn.Sigmoid()(eps_p/(pred+1e-6))
    p_aux_ = nn.Sigmoid()(eps_p_/(pred+1e-6))
#     loss = nn.BCELoss()(abs(p_aux),torch.ones_like(p_aux))+nn.BCELoss()(abs(p_aux_),torch.ones_like(p_aux_))
    loss = loss_function(p_aux,torch.ones_like(p_aux))+loss_function(p_aux_,torch.ones_like(p_aux_))
    loss.backward(retain_graph=True)

    # Based on perturbed image, get direction of greatest error
    eps_adv = eps.grad#/10**-learning_rate
    optimizer.zero_grad()
    # Use that direction as adversarial perturbation
    eps_adv = normalize_perturbation(eps_adv)
    d_adv = lamda * eps_adv.cuda()
    f_p = model(feature_only=True, feature1=f, feature2=f+d_adv)
    f_p_ = model(feature_only=True, feature1=f, feature2=f-d_adv)
    p = nn.Sigmoid()(f_p/(pred+1e-6))
    p_ = nn.Sigmoid()(f_p_/(pred+1e-6))
    vat_loss = loss_function(p,torch.ones_like(p))+loss_function(p_,torch.ones_like(p_))
    return eps_adv, d_adv, vat_loss


def d_loss(f, pred, model, y_val):
    diff_loss = 0
    length = len(pred)
    for i in range(length):
        for j in range(length):
            if j == i:
                continue
            pred_diff = model(feature_only=True, feature1=f[i], feature2=f[j])
            true_diff = y_val[i] - y_val[j]
            diff_loss += loss_function(pred_diff, torch.Tensor([true_diff]).view(-1,1))
    diff_loss = diff_loss/(length*(length-1))
    return diff_loss

In [14]:
def train(model, dataset, test_df, optimizer, loss_function, epoch):
    model.train()
    np.random.seed(epoch)
    max_len = np.max([len(dataset),len(test_df)])
    valList = np.arange(0,max_len)
    #shuffle them
    np.random.shuffle(valList)
    batch_list = []
    for i in range(0, max_len, batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch)
    for counter, batch in enumerate(batch_list):
        batch_df = dataset.loc[batch%len(dataset),:]
        batch_test = test_df.loc[batch%len(test_df),:]
        global_step = epoch * len(batch_list) + counter
        smiles_list = batch_df.cano_smiles.values
        smiles_list_test = batch_test.cano_smiles.values
        y_val = batch_df[tasks[0]].values.astype(float)
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)
        x_atom_test, x_bonds_test, x_atom_index_test, x_bond_index_test, x_mask_test, smiles_to_rdkit_list_test = get_smiles_array(smiles_list_test,feature_dicts)
        mol_feature, mol_prediction = model(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),output_feature=True)
        mol_feature_test, mol_prediction_test = model(torch.Tensor(x_atom_test),torch.Tensor(x_bonds_test),torch.cuda.LongTensor(x_atom_index_test),torch.cuda.LongTensor(x_bond_index_test),torch.Tensor(x_mask_test),output_feature=True)
        regression_loss = loss_function(mol_prediction, torch.Tensor(y_val).view(-1,1))
        eps_adv, d_adv, vat_loss = perturb_feature(mol_feature, mol_prediction, model, alpha=1, lamda=10**-learning_rate) # 10**-learning_rate     
#         diff_loss = d_loss(mol_feature, mol_prediction, model, y_val)
        eps_test, d_test, test_vat_loss = perturb_feature(mol_feature_test, mol_prediction_test, model, alpha=1, lamda=10**-learning_rate)
        if vat_loss>1 or test_vat_loss>1:
            vat_loss = 1*(vat_loss/(vat_loss+1e-6).item())
            test_vat_loss = 1*(test_vat_loss/(test_vat_loss+1e-6).item())
        
        logger.add_scalar('loss/regression', regression_loss, global_step)
#         logger.add_scalar('loss/diff_loss', diff_loss, global_step)
        logger.add_scalar('loss/VAT_Source', vat_loss, global_step)
        logger.add_scalar('loss/VAT', test_vat_loss, global_step)
    
        optimizer.zero_grad()
        loss = regression_loss + vat_loss + test_vat_loss
        loss.backward()
        optimizer.step()
        
def eval(model, dataset, output_feature=False):
    model.eval()
    predict_list = []
    test_MSE_list = []
    r2_list = []
    valList = np.arange(0,dataset.shape[0])
    batch_list = []
    feature_list = []
    d_list = []
    
# #     取dataset中排序后的第k个
#     sorted_dataset = dataset.sort_values(by=tasks[0],ascending=False)
#     k_df = sorted_dataset.iloc[[k-1]]
#     k_smiles = k_df['cano_smiles'].values
#     k_value = k_df[tasks[0]].values.astype(float)    
    
    for i in range(0, dataset.shape[0], batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch) 
#     print(batch_list)
    for counter, batch in enumerate(batch_list):
#         print(type(batch))
        batch_df = dataset.loc[batch,:]
        smiles_list = batch_df.cano_smiles.values
#         print(batch_df)
        y_val = batch_df[tasks[0]].values
#         print(type(y_val))
        
    
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)
        mol_feature, mol_prediction = model(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),output_feature=True)
        eps_adv, d_adv, vat_loss = perturb_feature(mol_feature, mol_prediction, model, alpha=1, lamda=10**-learning_rate)
        d = d_adv.cpu().detach().numpy().tolist()
        d_list.extend(d)
        mol_feature_output = mol_feature.cpu().detach().numpy().tolist()
        feature_list.extend(mol_feature_output)
#         MAE = F.l1_loss(mol_prediction, torch.Tensor(y_val).view(-1,1), reduction='none')   
#         print(type(mol_prediction))
        
        MSE = F.mse_loss(mol_prediction, torch.Tensor(y_val).view(-1,1), reduction='none')
#         r2 = caculate_r2(mol_prediction, torch.Tensor(y_val).view(-1,1))
# #         r2_list.extend(r2.cpu().detach().numpy())
#         if r2!=r2:
#             r2 = torch.tensor(0)
#         r2_list.append(r2.item())
#         predict_list.extend(mol_prediction.cpu().detach().numpy())
#         print(x_mask[:2],atoms_prediction.shape, mol_prediction,MSE)
        predict_list.extend(mol_prediction.cpu().detach().numpy())
#         test_MAE_list.extend(MAE.data.squeeze().cpu().numpy())
        test_MSE_list.extend(MSE.data.view(-1,1).cpu().numpy())
#     print(r2_list)
    if output_feature:
        return d_list, feature_list,caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list
    return caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list


In [15]:
import datetime
from tensorboardX import SummaryWriter
now = datetime.datetime.now().strftime('%b%d_%H-%M-%S')
log_dir = f'log/{"1_sim_"+file1+"_theorem"}'
if os.path.isdir(log_dir):
    for files in os.listdir(log_dir):
        os.remove(log_dir+"/"+files)
    os.rmdir(log_dir)
logger = SummaryWriter(log_dir)
print(log_dir)

log/1_sim_Kd_P47747_theorem


In [16]:
import scipy
train_f_list=[]
train_mse_list=[]
train_r2_list=[]
test_f_list=[]
test_mse_list=[]
test_r2_list=[]
val_f_list=[]
val_mse_list=[]
val_r2_list=[]
epoch_list=[]
train_predict_list=[]
test_predict_list=[]
val_predict_list=[]
train_y_list=[]
test_y_list=[]
val_y_list=[]
train_d_list=[]
test_d_list=[]
val_d_list=[]

batch_size = 10
epoch = 0

max_epoch = 1000
stopper = EarlyStopping(mode='higher', patience=30, filename=model_file + '_model.pth')
while epoch < max_epoch:
    train(model, train_df, test_df, optimizer, loss_function, epoch)
#     print(train_df.shape,test_df.shape)
    train_d, train_f, train_r2, train_MSE, train_predict = eval(model, train_df,output_feature=True)
    train_predict = np.array(train_predict)
    val_d, val_f, val_r2, val_MSE, val_predict = eval(model, val_df,output_feature=True)
    val_predict = np.array(val_predict)
    val_WTI = weighted_top_index(val_df, val_predict, len(val_df))
    val_AP = AP(val_df, val_predict, len(val_df))
    val_tau, _ = scipy.stats.kendalltau(val_predict,val_df[tasks[0]].values.astype(float).tolist())
    
    test_d, test_f, test_r2, test_MSE, test_predict = eval(model, test_df,output_feature=True)
    test_predict = np.array(test_predict)
    test_WTI = weighted_top_index(test_df, test_predict, len(test_df))
    test_AP = AP(test_df, test_predict, test_active)
    test_tau, _ = scipy.stats.kendalltau(test_predict,test_df[tasks[0]].values.astype(float).tolist())
    
    epoch = epoch + 1
    global_step = epoch * int(np.max([len(train_df),len(test_df)])/batch_size)
    logger.add_scalar('val/WTI', val_WTI, global_step)
    logger.add_scalar('val/AP', val_AP, global_step)
    logger.add_scalar('val/r2', val_r2, global_step)
    logger.add_scalar('val/RMSE', val_MSE**0.5, global_step)
    logger.add_scalar('val/Tau', val_tau, global_step)
    logger.add_scalar('test/TAP', test_AP, global_step)
    logger.add_scalar('test/Tau', test_tau, global_step)
    
    train_mse_list.append(train_MSE**0.5)
    train_r2_list.append(train_r2)
    val_mse_list.append(val_MSE**0.5)  
    val_r2_list.append(val_r2)
    train_f_list.append(train_f)
    val_f_list.append(val_f)
    test_f_list.append(test_f)
    epoch_list.append(epoch)
    train_predict_list.append(train_predict.flatten())
    test_predict_list.append(test_predict.flatten())
    val_predict_list.append(val_predict.flatten())
    train_y_list.append(train_df[tasks[0]].values)
    val_y_list.append(val_df[tasks[0]].values)
    test_y_list.append(test_df[tasks[0]].values)
    train_d_list.append(train_d)
    val_d_list.append(val_d)
    test_d_list.append(test_d)

    early_stop = stopper.step(val_r2, model)
#     print('epoch {:d}/{:d}, validation {} {:.4f}, {} {:.4f},best validation {r2} {:.4f}'.format(epoch, total_epoch, 'r2', val_r2, 'mse:',val_MSE, stopper.best_score))
    print('epoch:',epoch, 'R2:%.4f'%train_r2,'%.4f'%val_r2, 'RMSE:%.4f'%train_MSE**0.5, '%.4f'%val_MSE**0.5, 'WTI:%.4f'%val_WTI,'%.4f'%test_WTI,'AP:%.4f'%test_AP, 'Tau:%.4f'%val_tau,'%.4f'%test_tau)
    if early_stop:
        break

stopper.load_checkpoint(model)

    
test_r2, test_MSE, test_predict = eval(model, test_df)
test_r2_a, test_MSE_a, test_predict_a = eval(model, test_df[:test_active])
test_r2_ina, test_MSE_ina, test_predict_ina = eval(model, test_df[test_active:].reset_index(drop=True))
    
test_predict = np.array(test_predict)
test_tau, _ = scipy.stats.kendalltau(test_predict,test_df[tasks[0]].values.astype(float).tolist())

k_list = [int(len(test_df)*0.05),int(len(test_df)*0.1),int(len(test_df)*0.15),int(len(test_df)*0.2),int(len(test_df)*0.25),
          int(len(test_df)*0.3),int(len(test_df)*0.4),int(len(test_df)*0.5),50,100,150,200,250,300]
topk_list =[]
false_positive_rate_list = []
for k in k_list:
    a,b = topk_acc_recall(test_df, test_predict, k, test_active, False, epoch)
    topk_list.append(a)
    false_positive_rate_list.append(b)
WTI = weighted_top_index(test_df, test_predict, test_active)
ap = AP(test_df, test_predict, test_active)

print("----------")
print(epoch, 'R2:%.4f'%train_r2,'%.4f'%val_r2,'%.4f'%test_r2_a, 'RMSE:%.4f'%train_MSE**0.5, '%.4f'%val_MSE**0.5, '%.4f'%test_MSE_a**0.5,'WTI:%.4f'%WTI,'AP:%.4f'%ap,'Tau:%.4f'%test_tau)


epoch: 1 R2:0.0573 0.0100 RMSE:1.0484 1.6004 WTI:0.1047 0.2199 AP:0.3475 Tau:-0.2000 -0.2203
EarlyStopping counter: 1 out of 30
epoch: 2 R2:0.0596 0.0096 RMSE:1.0661 1.7407 WTI:0.1047 0.2204 AP:0.3475 Tau:-0.2000 -0.2188
epoch: 3 R2:0.0625 0.0110 RMSE:0.9526 1.5351 WTI:0.1047 0.2205 AP:0.3475 Tau:-0.2000 -0.2211
epoch: 4 R2:0.0655 0.0116 RMSE:0.9422 1.5185 WTI:0.1047 0.2208 AP:0.3475 Tau:-0.2000 -0.2219
epoch: 5 R2:0.0694 0.0126 RMSE:0.9299 1.4761 WTI:0.1047 0.2211 AP:0.3475 Tau:-0.2000 -0.2229
epoch: 6 R2:0.0747 0.0134 RMSE:0.9092 1.4225 WTI:0.1047 0.2212 AP:0.3475 Tau:-0.2000 -0.2224
epoch: 7 R2:0.0802 0.0143 RMSE:0.8976 1.3817 WTI:0.1267 0.2212 AP:0.3475 Tau:-0.1636 -0.2245
epoch: 8 R2:0.0878 0.0147 RMSE:0.8854 1.3413 WTI:0.1267 0.2219 AP:0.3475 Tau:-0.1636 -0.2291
epoch: 9 R2:0.0962 0.0151 RMSE:0.8781 1.3035 WTI:0.1366 0.2220 AP:0.3475 Tau:-0.1273 -0.2309
epoch: 10 R2:0.1084 0.0151 RMSE:0.8670 1.2653 WTI:0.1738 0.2223 AP:0.3475 Tau:-0.0909 -0.2332
EarlyStopping counter: 1 out of 30

EarlyStopping counter: 1 out of 30
epoch: 73 R2:0.7714 0.7457 RMSE:0.4423 0.7205 WTI:0.4060 0.3329 AP:0.4890 Tau:0.7455 0.2165
EarlyStopping counter: 2 out of 30
epoch: 74 R2:0.7717 0.7577 RMSE:0.4866 0.6115 WTI:0.4129 0.3299 AP:0.4749 Tau:0.7455 0.2021
EarlyStopping counter: 3 out of 30
epoch: 75 R2:0.7741 0.7335 RMSE:0.4567 0.6561 WTI:0.4060 0.3309 AP:0.4859 Tau:0.7455 0.2098
EarlyStopping counter: 4 out of 30
epoch: 76 R2:0.7752 0.7301 RMSE:0.4458 0.6768 WTI:0.4060 0.3325 AP:0.4914 Tau:0.7455 0.2173
EarlyStopping counter: 5 out of 30
epoch: 77 R2:0.7770 0.7232 RMSE:0.4679 0.6421 WTI:0.4060 0.3333 AP:0.4937 Tau:0.7455 0.2247
EarlyStopping counter: 6 out of 30
epoch: 78 R2:0.7785 0.7472 RMSE:0.4572 0.6340 WTI:0.4060 0.3336 AP:0.4910 Tau:0.7455 0.2206
EarlyStopping counter: 7 out of 30
epoch: 79 R2:0.7740 0.6859 RMSE:0.4735 0.8591 WTI:0.4281 0.3385 AP:0.5088 Tau:0.7818 0.2673
EarlyStopping counter: 8 out of 30
epoch: 80 R2:0.7752 0.6925 RMSE:0.4996 0.8993 WTI:0.4281 0.3388 AP:0.5082 Ta

EarlyStopping counter: 3 out of 30
epoch: 140 R2:0.8530 0.7896 RMSE:0.3614 0.5768 WTI:0.4160 0.3647 AP:0.5776 Tau:0.7818 0.3508
EarlyStopping counter: 4 out of 30
epoch: 141 R2:0.8547 0.7969 RMSE:0.3561 0.5935 WTI:0.4160 0.3650 AP:0.5799 Tau:0.7818 0.3480
EarlyStopping counter: 5 out of 30
epoch: 142 R2:0.8562 0.8041 RMSE:0.3567 0.5533 WTI:0.3788 0.3642 AP:0.5856 Tau:0.7455 0.3444
EarlyStopping counter: 6 out of 30
epoch: 143 R2:0.8575 0.8063 RMSE:0.3689 0.5301 WTI:0.4160 0.3665 AP:0.5979 Tau:0.7818 0.3549
epoch: 144 R2:0.8579 0.8210 RMSE:0.3790 0.4942 WTI:0.3788 0.3630 AP:0.5811 Tau:0.7455 0.3442
EarlyStopping counter: 1 out of 30
epoch: 145 R2:0.8576 0.8210 RMSE:0.3921 0.4801 WTI:0.3788 0.3585 AP:0.5644 Tau:0.7455 0.3326
EarlyStopping counter: 2 out of 30
epoch: 146 R2:0.8611 0.8124 RMSE:0.3476 0.5446 WTI:0.3788 0.3656 AP:0.5920 Tau:0.7455 0.3526
epoch: 147 R2:0.8622 0.8243 RMSE:0.3602 0.5077 WTI:0.3788 0.3659 AP:0.5925 Tau:0.7455 0.3539
epoch: 148 R2:0.8608 0.8316 RMSE:0.3792 0.4725

EarlyStopping counter: 1 out of 30
epoch: 207 R2:0.9191 0.8746 RMSE:0.2872 0.3886 WTI:0.3788 0.3780 AP:0.6723 Tau:0.7455 0.4452
epoch: 208 R2:0.9201 0.8914 RMSE:0.2716 0.3684 WTI:0.3835 0.3795 AP:0.6699 Tau:0.7818 0.4419
EarlyStopping counter: 1 out of 30
epoch: 209 R2:0.9195 0.8818 RMSE:0.3197 0.3686 WTI:0.3788 0.3779 AP:0.6703 Tau:0.7455 0.4526
EarlyStopping counter: 2 out of 30
epoch: 210 R2:0.9216 0.8819 RMSE:0.2692 0.3794 WTI:0.3835 0.3788 AP:0.6709 Tau:0.7818 0.4416
EarlyStopping counter: 3 out of 30
epoch: 211 R2:0.9233 0.8812 RMSE:0.2735 0.3837 WTI:0.3788 0.3815 AP:0.6735 Tau:0.7455 0.4562
EarlyStopping counter: 4 out of 30
epoch: 212 R2:0.9232 0.8882 RMSE:0.3114 0.3619 WTI:0.3835 0.3797 AP:0.6746 Tau:0.7818 0.4549
EarlyStopping counter: 5 out of 30
epoch: 213 R2:0.9225 0.8845 RMSE:0.2604 0.4287 WTI:0.3788 0.3830 AP:0.6960 Tau:0.7455 0.4618
EarlyStopping counter: 6 out of 30
epoch: 214 R2:0.9230 0.8823 RMSE:0.3887 0.3875 WTI:0.3835 0.3802 AP:0.6679 Tau:0.7818 0.4442
EarlyStoppi

EarlyStopping counter: 12 out of 30
epoch: 273 R2:0.9661 0.9072 RMSE:0.1733 0.3252 WTI:0.3853 0.3891 AP:0.7191 Tau:0.8545 0.5111
EarlyStopping counter: 13 out of 30
epoch: 274 R2:0.9567 0.8891 RMSE:0.2216 0.3740 WTI:0.4160 0.3922 AP:0.7300 Tau:0.7818 0.5247
EarlyStopping counter: 14 out of 30
epoch: 275 R2:0.9593 0.8863 RMSE:0.3223 0.4431 WTI:0.3845 0.3862 AP:0.7091 Tau:0.8182 0.4967
EarlyStopping counter: 15 out of 30
epoch: 276 R2:0.9672 0.8995 RMSE:0.2044 0.3652 WTI:0.3835 0.3911 AP:0.7265 Tau:0.8182 0.5193
EarlyStopping counter: 16 out of 30
epoch: 277 R2:0.9660 0.9076 RMSE:0.3725 0.4519 WTI:0.3853 0.3880 AP:0.7106 Tau:0.8545 0.5039
EarlyStopping counter: 17 out of 30
epoch: 278 R2:0.9635 0.9023 RMSE:0.1885 0.3343 WTI:0.3835 0.3870 AP:0.7090 Tau:0.8182 0.5200
EarlyStopping counter: 18 out of 30
epoch: 279 R2:0.9688 0.9150 RMSE:0.1813 0.3180 WTI:0.3853 0.3866 AP:0.7198 Tau:0.8545 0.5049
EarlyStopping counter: 19 out of 30
epoch: 280 R2:0.9696 0.9098 RMSE:0.1622 0.3219 WTI:0.3835 0.3

In [17]:
print(' epoch:',epoch,'r2:%.4f'%test_r2_a,'RMSE:%.4f'%test_MSE_a**0.5,'WTI:%.4f'%WTI,'AP:%.4f'%ap,'Tau:%.4f'%test_tau,'\n','\n',
      'Top-5:%.4f'%topk_list[0],'Top-5-fp:%.4f'%false_positive_rate_list[0],'\n',
      'Top-10:%.4f'%topk_list[1],'Top-10-fp:%.4f'%false_positive_rate_list[1],'\n',
      'Top-15:%.4f'%topk_list[2],'Top-15-fp:%.4f'%false_positive_rate_list[2],'\n',
      'Top-20:%.4f'%topk_list[3],'Top-20-fp:%.4f'%false_positive_rate_list[3],'\n',
      'Top-25:%.4f'%topk_list[4],'Top-25-fp:%.4f'%false_positive_rate_list[4],'\n',
      'Top-30:%.4f'%topk_list[5],'Top-30-fp:%.4f'%false_positive_rate_list[5],'\n',
      'Top-40:%.4f'%topk_list[6],'Top-40-fp:%.4f'%false_positive_rate_list[6],'\n',
      'Top-50:%.4f'%topk_list[7],'Top-50-fp:%.4f'%false_positive_rate_list[7],'\n','\n',
      'Top50:%.4f'%topk_list[8],'Top50-fp:%.4f'%false_positive_rate_list[8],'\n',
      'Top100:%.4f'%topk_list[9],'Top100-fp:%.4f'%false_positive_rate_list[9],'\n',
      'Top150:%.4f'%topk_list[10],'Top150-fp:%.4f'%false_positive_rate_list[10],'\n',
      'Top200:%.4f'%topk_list[11],'Top200-fp:%.4f'%false_positive_rate_list[11],'\n',
      'Top250:%.4f'%topk_list[12],'Top250-fp:%.4f'%false_positive_rate_list[12],'\n',
      'Top300:%.4f'%topk_list[13],'Top300-fp:%.4f'%false_positive_rate_list[13],'\n')

 epoch: 291 r2:0.5023 RMSE:0.8083 WTI:0.3526 AP:0.7113 Tau:0.5003 
 
 Top-5:0.4286 Top-5-fp:0.0000 
 Top-10:0.6429 Top-10-fp:0.0000 
 Top-15:0.7619 Top-15-fp:0.0000 
 Top-20:0.7143 Top-20-fp:0.1071 
 Top-25:0.6944 Top-25-fp:0.2500 
 Top-30:0.6977 Top-30-fp:0.2791 
 Top-40:0.7200 Top-40-fp:0.3684 
 Top-50:0.8200 Top-50-fp:0.4306 
 
 Top50:0.6600 Top50-fp:0.3400 
 Top100:0.9400 Top100-fp:0.5300 
 Top150:1.0000 Top150-fp:0.6267 
 Top200:1.0000 Top200-fp:0.4700 
 Top250:1.0000 Top250-fp:0.3760 
 Top300:1.0000 Top300-fp:0.3133 



In [18]:
print('target_file:',raw_filename)
print('inactive_file:',inactive_filename)
np.savez('/data/yinmingyue/Sim/code/result/Sim+/0_Sim_Space_'+file1, epoch_list, train_f_list, train_d_list, 
         train_predict_list, train_y_list, val_f_list, val_d_list, val_predict_list, val_y_list, test_f_list, 
         test_d_list, test_predict_list, test_y_list)
sim_space = np.load('/data/yinmingyue/Sim/code/result/Sim+/0_Sim_Space_'+file1+".npz")
print(sim_space['arr_10'].shape)

target_file: ../data/active_inactive/median_active/Kd/P47747.csv
inactive_file: ../data/active_inactive/median_inactive/Kd/P47747.csv
(291, 144, 100)


In [19]:
test_df.to_csv(path_or_buf='/data/yinmingyue/Sim/code/result/Sim+/Sim_'+file1+'.csv')
print(test_df[-5:].head())

                                                smiles  value  \
139  C1=CC(=CC=C1C(=O)NN)OCCCC2=CN=CN2.C(=CC(=O)O)C...   -4.1   
140                              CC1=C(SC(=N1)C)CCN.Br   -4.1   
141  C1=CC(=CC=C1C(=O)N)OCCCC2=CN=CN2.C(=CC(=O)O)C(...   -4.1   
142                    CC(C1=CC=C(C=C1)OCCCC2=CN=CN2)O   -4.1   
143                           CSC(=NCCCC1=CN=CN1)N.I.I   -4.1   

                                         cano_smiles   predict  
139  NNC(=O)c1ccc(OCCCc2cnc[nH]2)cc1.O=C(O)C=CC(=O)O -4.901513  
140                              Br.Cc1nc(C)c(CCN)s1 -6.510898  
141   NC(=O)c1ccc(OCCCc2cnc[nH]2)cc1.O=C(O)C=CC(=O)O -5.084274  
142                    CC(O)c1ccc(OCCCc2cnc[nH]2)cc1 -4.661007  
143                        CSC(N)=NCCCc1cnc[nH]1.I.I -4.053786  
